In [1]:
!pip install llama-index transformers accelerate bitsandbytes
!pip install llama-index-llms-huggingface

!pip install docx2txt

!pip install pypdf==4.0.1

!pip install cryptography
!pip install llama-index-embeddings-huggingface
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 35.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  

In [2]:
from cryptography.fernet import Fernet, InvalidToken
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.core import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader, ServiceContext, StorageContext, load_index_from_storage

# Configure quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Define a function to convert messages to a prompt
def messages_to_prompt(messages):
    prompt = "\n".join([f"\n{message.content}\n" for message in messages if message.role in ['system', 'user', 'assistant']])
    prompt = "\n\n" + prompt if not prompt.startswith("\n") else prompt
    return prompt + "\n"

# Initialize HuggingFaceLLM
llm = HuggingFaceLLM(
    model_name="stabilityai/stablelm-zephyr-3b",
    tokenizer_name="stabilityai/stablelm-zephyr-3b",
    query_wrapper_prompt=PromptTemplate("\n\n\n{query_str}\n\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    generate_kwargs={"temperature": 0.3},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

model = AutoModelForCausalLM.from_pretrained(
    'stabilityai/stablelm-zephyr-3b',
    trust_remote_code=True,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import base64

# Generate a key for encryption
key = get_random_bytes(16)
print(key)
cipher = AES.new(key, AES.MODE_ECB)

b"\xa6\xdc\xd2\xd7\x03\x02\x9b\xedz\x9f\xa9'\xb8k\xb7\x90"


In [33]:
# Function to encrypt the user input
def encrypt_input(user_input):
    if user_input is None:
        return None
    user_input = user_input.encode()  # Convert to bytes
    ciphered_data = cipher.encrypt(pad(user_input, AES.block_size))  # Pad the input and then encrypt
    return base64.b64encode(ciphered_data).decode()  # Encode to base64 and then decode to string

def decrypt_input(encrypted_input):
    if encrypted_input is None:
        return None
    try:
        encrypted_input = base64.b64decode(encrypted_input.encode())  # Encode to bytes and then decode from base64
        decipher = AES.new(key, AES.MODE_ECB)  # Create a new decipher
        plain_text = unpad(decipher.decrypt(encrypted_input), AES.block_size)  # Decrypt and then unpad the data
        return plain_text.decode()
    except (ValueError, KeyError) as e:
        print(f"Error during decryption: {str(e)}")
        return None

In [6]:
!pip install llama-index-embeddings-huggingface


In [8]:
from llama_index.core import ServiceContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core import ServiceContext
documents = SimpleDirectoryReader("files").load_data()
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
vector_index.storage_context.persist("index")

from llama_index.core import StorageContext, load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="index")
index = load_index_from_storage(storage_context=storage_context, service_context=service_context)


<ipython-input-8-61d03047d5b9>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")


In [45]:
# Create a Query Engine
query_engine = index.as_query_engine(similarity_top_k=5)

def ask_question(query):
    response = query_engine.query(query)
    if response.response:
        # Pad the response to match the block size
        response.response = base64.b64encode(cipher.encrypt(pad(response.response.encode(), AES.block_size))).decode()
    return response.response

def ask_questions_interactively():
    while True:
        user_input = input("Ask a question (type 'exit' to stop): ")
        if user_input.lower() == 'exit':
            print("Exiting the interactive question mode.")
            break
        else:
            encrypted_input = encrypt_input(user_input)
            print("Encrypted input:",encrypted_input)
            decrypted_result = decrypt_input(encrypted_input)
            print("Decrypted input:",decrypted_result)
            query_result = ask_question(decrypted_result)
            if query_result:
                print("Response:", decrypt_input(query_result))

# Call the interactive function
ask_questions_interactively()

Ask a question (type 'exit' to stop): what is in module 2


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Encrypted input: t38+ClOyCgJg5OXpb+qr394QKIvEnugt2uVOC31k794=
Decrypted input: what is in module 2
Response: 
Module 2 - Symmetric Encryption Algorithms
Module 2 title is "Symmetric Encryption Algorithms". This module covers the following topics:

* Introduction to Stream cipher
* Block cipher: DES, AES, IDEA, Block Cipher Operation
* Random Bit Generation and RC4

These topics focus on the symmetric key cryptographic techniques, which are commonly used in encryption systems. The module aims to provide an understanding of the underlying concepts and techniques used in symmetric encryption algorithms.
Ask a question (type 'exit' to stop): title of the document


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Encrypted input: UeV2rnhaYnDd5qyjV+eEZ+bKRS40/f8vQB5Wkxb+/1I=
Decrypted input: title of the document
Response: 
The title of the document is "BCSE309L Cryptography and Network Security". This is derived from the context information where it is mentioned that this is the course code and name for the course being taught, along with the module topics and the text book used.
Ask a question (type 'exit' to stop): exit
Exiting the interactive question mode.


In [21]:
# Example of asking a question
query_result = ask_question("What are the countries given?")
print(query_result)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



There is no information about the countries given in the provided context.
